In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('..\\Bases\\Criados\\base_doencas_cardiacas.csv')

In [3]:
df.columns[df.isnull().any()]

Index(['D00901', 'D014', 'E006011', 'E008', 'E01602', 'E017', 'E01802', 'E019',
       'F001021', 'F007021', 'F008021', 'VDF00102', 'G060', 'G062', 'I006',
       'I004', 'I00401', 'I00402', 'I00403', 'J00402', 'J00404', 'J00801',
       'M00203', 'M011011', 'M011021', 'M011051', 'M011071', 'N004', 'N005',
       'N008', 'P00104', 'P00404', 'P02801', 'P029', 'P035', 'P03701',
       'P03702', 'P051', 'P052', 'P053', 'P05401', 'P05404', 'P05407',
       'P05410', 'P05413', 'P05416', 'P05419', 'Q00201', 'Q03001', 'Q060',
       'Q064', 'Q06506', 'Q06507', 'Q06508', 'Q06509', 'Q06601', 'Q067',
       'H003', 'H004', 'H010', 'D00301', 'D008', 'E002', 'E003', 'E004',
       'E005', 'E011', 'E014011', 'E01601', 'E01603', 'E01605', 'E01801',
       'I00103', 'I001031', 'P00103', 'P00403', 'P05901', 'P05902', 'P05903',
       'P05904', 'Q06307', 'Q06308', 'Q06309', 'Q06310', 'Q06311', 'H002'],
      dtype='object')

In [4]:
(df.isnull().sum() > 0).sum()

np.int64(86)

### Escolaridade

Atributos utilizados:
1. D00201: se frequenta escola atualmente
2. D00301: qual curso frequenta
3. D008: se já frequentou escola ou creche
4. D00901: Qual foi o curso mais elevado que frequentou?
5. D014: se concluiu esse curso

In [5]:
def categorizar_escolaridade(row):
    if row['D00201'] == 1:
        return 'Frequenta'
    elif row['D00201'] == 2:
        if row['D008'] == 1:
            return 'Frequentou'
        elif row['D008'] == 2:
            return 'Não frequentou'
    return pd.NA  # Caso não se encaixe em nenhuma condição

df['Situacao_escolar'] = df.apply(categorizar_escolaridade, axis=1)

print(df['Situacao_escolar'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Situacao_escolar'].isna().sum())


Situacao_escolar
Frequentou        20923
Não frequentou     2741
Frequenta           687
Name: count, dtype: int64
Valores faltantes: 0


In [6]:
df['D00301'].value_counts()

D00301
8.0     214
4.0     115
9.0     114
5.0      96
3.0      54
6.0      44
7.0      22
10.0     19
11.0      9
Name: count, dtype: int64

In [7]:
def map_escolaridade(row): 
    situacao = row['Situacao_escolar']
    curso = row['D00901']
    concluiu = row['D014']
    curso_atual = row['D00301']

    # Se nunca frequentou escola, categoria = 0
    if situacao == 'Não frequentou':
        return 0

    # Usa D00901 (curso mais elevado) prioritariamente
    if curso in [1, 2]:  # Creche, Pré-escola
        categoria = 1
    elif curso in [3, 4, 5, 6, 7, 8]:  # Alfabetização, primário, EJA fundamental etc.
        categoria = 2
    elif curso in [9, 10, 11]:  # Ensino médio, EJA médio
        categoria = 3
    elif curso == 12:  # Superior
        categoria = 4
    elif curso in [13, 14, 15]:  # Pós-graduação
        categoria = 5
    # Se curso é inválido ou ausente, tenta usar curso_atual (D00301)
    elif curso_atual in [3, 4, 5]:
        categoria = 1  # Educação infantil
    elif curso_atual in [6, 7]:
        categoria = 2  # Fundamental
    elif curso_atual in [8, 9]:
        categoria = 3  # Médio
    elif curso_atual in [10, 11]:
        categoria = 4  # Superior
    else:
        return np.nan

    # Rebaixar se não concluiu (exceto se usou curso_atual)
    if concluiu == 2 and curso in range(1, 16):
        categoria -= 1
        if categoria < 0:
            categoria = 0

    return categoria



df['Escolaridade'] = df.apply(map_escolaridade, axis=1)

print(df['Escolaridade'].value_counts(dropna=False))

# 0: Nunca frequentou escola
# 1: Educação infantil
# 2: Ensino fundamental
# 3: Ensino médio
# 4: Ensino superior
# 5: Pós-graduação

Escolaridade
1    9443
3    5266
2    4133
0    2741
4    1828
5     940
Name: count, dtype: int64


In [8]:
df.drop(columns=['D00201', 'D00301', 'D008', 'D00901', 'D014', 'Situacao_escolar'], inplace=True)

### Renda

1. E014011: se era trabalhador não remunerado ajudando parente/domicílio
2. E01601: se recebia dinheiro no trabalho principal
3. E01602: valor da renda do trabalho principal
4. E01801: se recebia dinheiro em outro(s) trabalho(s)
5. E01802: valor da renda dos outros trabalhos

Regras:

1. Se E01601 != 1 ou E01801 != 1, a renda respectiva é 0;
2. Se a pessoa é não remunerada (E014011 = 1, 2, 3), renda é 0;
3. Se E01602 ou E01802 estiverem NaN, mas a pessoa disse que recebia (E01601 ou E01801 = 1), considerar como NaN final.
4. Renda total é a soma de E01602 e E01802.

In [9]:
def calcular_renda(row):
    # Regra 2: trabalhador não remunerado (1, 2 ou 3)
    if row['E014011'] in [1, 2, 3]:
        return 0.0

    # Regra 1 + 3: verificar se recebia dinheiro
    if row['E01601'] == 1:
        renda1 = row['E01602'] if pd.notna(row['E01602']) else np.nan
    else:
        renda1 = 0.0

    if row['E01801'] == 1:
        renda2 = row['E01802'] if pd.notna(row['E01802']) else np.nan
    else:
        renda2 = 0.0

    # Regra 4: soma ajustada
    if pd.isna(renda1) or pd.isna(renda2):
        if pd.notna(renda1) and pd.isna(renda2):
            return renda1
        elif pd.notna(renda2) and pd.isna(renda1):
            return renda2
        else:
            return np.nan

    return renda1 + renda2

df['Renda'] = df.apply(calcular_renda, axis=1)
print(f"Valores faltantes:",df['Renda'].isna().sum())

Valores faltantes: 0


In [10]:
print((df['Renda'] == 0.0).sum())

12208


In [11]:
df.drop(columns=['E014011', 'E01601', 'E01602', 'E01801', 'E01802'], inplace=True)

### Horas trabalhadas *incompleto (checar quant trabalho 3. Três ou mais)

1. E011: número de trabalhos na semana 
2. E017: horas trabalhas em 1 trabalho
3. E019: horas trabalhas em mais de 1 trabalho

In [12]:
print("M00203 faltantes:", df['M00203'].isna().sum())
print(df['M00203'].value_counts(dropna=False)) #ver isso aqui (incompleto)

M00203 faltantes: 3
M00203
1.0    23962
2.0      383
NaN        3
9.0        3
Name: count, dtype: int64


In [13]:
def quantidade_trabalhos(row):
    if row['E011'] == 1:
        return 1
    elif row['E011'] == 2:
            return 2
    elif row['E011'] == 3:
            return 3
    else:
         return 0

df['Quant_trabalhos'] = df.apply(quantidade_trabalhos, axis=1)

print(df['Quant_trabalhos'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Quant_trabalhos'].isna().sum())

Quant_trabalhos
0    11977
1    11634
2      655
3       85
Name: count, dtype: int64
Valores faltantes: 0


In [14]:
print("E017 faltantes:", df['E017'].isna().sum())
print("E019 faltantes:", df['E019'].isna().sum())

E017 faltantes: 11977
E019 faltantes: 23611


In [15]:
df['Quant_trabalhos'] = df['Quant_trabalhos'].astype(int)
df.loc[df['Quant_trabalhos'] == 0, 'E017'] = 0
df.loc[df['Quant_trabalhos'].isin([0, 1]), 'E019'] = 0

#df['E017']=df['E017'].fillna(0)
#df['E019']=df['E019'].fillna(0)

In [16]:
print("E017 faltantes:", df['E017'].isna().sum())
print("E019 faltantes:", df['E019'].isna().sum())

E017 faltantes: 0
E019 faltantes: 0


In [17]:
df['Horas_trabalhadas'] = df['E017'] + df['E019']

print("Valores faltantes:", df['Horas_trabalhadas'].isna().sum())

Valores faltantes: 0


In [18]:
df['Quant_trabalhos'] = pd.to_numeric(df['Quant_trabalhos'], errors='coerce')
df[(df['Renda'] == 0.0) & (df['Quant_trabalhos'] == 0) & (df['Horas_trabalhadas'] == 0.0)][['Renda', 'Quant_trabalhos', 'Horas_trabalhadas']]

,Renda,Quant_trabalhos,Horas_trabalhadas
0,0.0,0,0.0
4,0.0,0,0.0
6,0.0,0,0.0
9,0.0,0,0.0
10,0.0,0,0.0
...,...,...,...
24314,0.0,0,0.0
24325,0.0,0,0.0
24326,0.0,0,0.0
24334,0.0,0,0.0


In [19]:
df.drop(columns=['E01603', 'E01605', 'E017', 'E019'], inplace=True)

### Afastamento do trabalho por motivo de saúde & motivo de saúde relacionado ao trabalho

1. E005: tinha algum trabalho remunerado do qual estava temporariamente afastado?
2. E006011: por que motivo estava afastado desse trabalho
3. E008: a doença ou acidente foi relacionado ao trabalho?

In [20]:
def temporariamente_afastado(row):
    if row['E005'] == 1:
        return 'Sim'
    elif row['E005'] == 2:
            return 'Não'
    else:
         return 'Não'

df['Temporariamente_afastado'] = df.apply(temporariamente_afastado, axis=1)


print(df['Temporariamente_afastado'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Temporariamente_afastado'].isna().sum())

Temporariamente_afastado
Não    24022
Sim      329
Name: count, dtype: int64
Valores faltantes: 0


In [21]:
df.loc[(df['Temporariamente_afastado']== 'Não'), 'E006011'] = 0 #se E006011=0, não esteve afastado

In [22]:
df['E006011'].isna().sum()

np.int64(0)

In [23]:
df.loc[(df['E006011'].isin([0, 1, 2, 4, 5, 6, 7])), 'E008'] = 0

In [24]:
df['E008'].isna().sum()

np.int64(0)

In [25]:
df.drop(columns=['E001','E002', 'E003', 'E004', 'E005'], inplace=True)

### Mobilidade

1. G059: usa algum aparelho de auxílio para se locomover?
2. G060: faz uso de cadeira de rodas?
3. G062: faz uso de bengala, muletas ou andador?

In [26]:
def classificar_mobilidade(row):
    usa_aparelho = row['G059'] == 1
    cadeira_rodas = row['G060'] == 1
    bengala_muleta = row['G062'] == 1

    if not usa_aparelho:
        return 0.0  # Não usa nenhum aparelho
    if cadeira_rodas:
        return 1.0  # Só cadeira de rodas
    if bengala_muleta:
        return 2.0  # Só bengala/muleta/andador
    if cadeira_rodas and bengala_muleta:
        return 3.0  # Usa os dois
    if not cadeira_rodas and not bengala_muleta:
        return 4.0  # Não usa nenhum dos dois (uso outras coisas)
    return np.nan  # Caso de inconsistência ou valores faltantes

df['Mobilidade_auxiliar'] = df.apply(classificar_mobilidade, axis=1)

# Visualizar resultado:
print(df['Mobilidade_auxiliar'].value_counts(dropna=False))
print(df['Mobilidade_auxiliar'].isna().sum())

Mobilidade_auxiliar
0.0    23631
2.0      517
1.0      149
4.0       54
Name: count, dtype: int64
0


In [27]:
df = df.drop(columns=['G060', 'G062'])

### Plano de Saúde

1. I00102: se tem plano
2. I006: avaliação do plano

In [28]:
def classificar_plano(row):
    tem_plano = row['I00102']
    avaliacao = row['I006']
    
    if tem_plano == 2:
        return 'Não tem plano'  # Não tem plano
    elif avaliacao in [1, 2]:
        return 'Bom/muito bom'  # Muito bom ou bom
    elif avaliacao == 3:
        return 'Regular'  # Regular
    elif avaliacao in [4, 5]:
        return 'Ruim/muito ruim' # Ruim ou muito ruim
    elif avaliacao == 6:
        return 'Nunca usou'  # Nunca usou
    else:
        return np.nan  # Valores faltantes ou inválidos

df['Avaliacao_plano'] = df.apply(classificar_plano, axis=1)

print(df['Avaliacao_plano'].value_counts(dropna=False))
print(df['Avaliacao_plano'].isna().sum())

# 0     | Não tem plano                
# 1     | Avaliação muito boa ou boa   
# 2     | Avaliação regular            
# 3     | Avaliação ruim ou muito ruim 
# 4     | Nunca usou o plano 

Avaliacao_plano
Não tem plano      19748
Bom/muito bom       3409
Regular              941
Ruim/muito ruim      157
Nunca usou            96
Name: count, dtype: int64
0


In [29]:
df = df.drop(columns=['I006'])

3. I004: se é de instituição pública
4. I00103: se é morador 
5. I001031: número do titular

*C00301 == I001031 --> é titular    

In [30]:
def definir_plano_publico(row):
    if row['I00102'] != 1:
        return 0.0  # Não tem plano
    if row['I00103'] != 1 and row['C00301'] != row['I001031']: # não é morador
        return 0.0  # não é titular do plano
    if row['I004'] == 1:
        return 1.0  # Plano é de instituição pública
    if row['I004'] == 2:
        return 2.0  # Plano não é de instituição pública
    
    return 0.0

df['Plano_publico'] = df.apply(definir_plano_publico, axis=1)

print(df['Plano_publico'].value_counts(dropna=False))
print(df['Plano_publico'].isna().sum())

#0     | Não tem plano e/ou não é morador + titular do plano              
#1     | Plano é de instituição pública   
#2     | Plano não é de instituição pública   

KeyError: 'C00301'

In [ ]:
df = df.drop(columns=['I004'])

6. I00401: plano dá direito a consultas
7. I00402: plano dá direito a exames
8. I00403: plano dá direito a internações

In [ ]:
def Cobertura_plano(row):
    # Se não tem plano, retorna 0.0
    if pd.isna(row['I00102']) or row['I00102'] != 1:
        return 0.0

    # Verifica titularidade
    if pd.isna(row['I00103']) or row['I00103'] != 1:
        return 0.0
    if pd.isna(row['C00301']) or pd.isna(row['I001031']) or row['C00301'] != row['I001031']:
        return 0.0

    # Verifica cobertura
    coberturas = [
        row['I00401'] == 1 if pd.notna(row['I00401']) else False,
        row['I00402'] == 1 if pd.notna(row['I00402']) else False,
        row['I00403'] == 1 if pd.notna(row['I00403']) else False
    ]
    total_cobertura = sum(coberturas)

    if total_cobertura == 1:
        return 1.0
    elif total_cobertura == 2:
        return 2.0
    elif total_cobertura == 3:
        return 3.0
    elif total_cobertura == 0:
        return 4.0  # Não cobre nada, apesar de ter plano e titular
    else:
        return np.nan  # Caso de erro

df['Cobertura_plano'] = df.apply(Cobertura_plano, axis=1)

print(df['Cobertura_plano'].value_counts(dropna=False))
print("Valores nulos:", df['Cobertura_plano'].isna().sum())


# 1- se o plano cobre uma das coisas
# 2- se o plano cobre duas das coisas
# 3- se o plano cobre as três coisas

KeyError: 'C00301'

In [ ]:
df = df.drop(columns=['I00102', 'I00401', 'I00402', 'I00403', 'I00103', 'I001031'])

### Preenchendo aqui os atributos J00402, J00404 e J00801

In [31]:
df.loc[df['J002'] == 2, 'J00402'] = 0
df.loc[df['J002'] == 2, 'J00404'] = 0

In [32]:
print(df['J00402'].isna().sum())
print(df['J00404'].isna().sum())

0
0


In [33]:
df.loc[df['J007'] == 2, 'J00801'] = 0
print(df['J00801'].isna().sum())

0


### Exposição tóxica no trabalho

Atributos:
1. M011011: substâncias químicas
2. M011021: barulho intenso (ruído)
3. M011051: lixo (resíduo urbano)
4. M011071: poeira mineral

In [34]:
def marcar_exposicao_quimica(row):
    trabalho = row['E011']
    exposicao = row['M011011']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_quimica'] = df.apply(marcar_exposicao_quimica, axis=1)

print(df['Exposicao_quimica'].value_counts(dropna=False))
print(df['Exposicao_quimica'].isna().sum())

Exposicao_quimica
2    22934
1     1417
Name: count, dtype: int64
0


In [35]:
def marcar_exposicao_ruido(row):
    trabalho = row['E011']
    exposicao = row['M011021']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_ruido'] = df.apply(marcar_exposicao_ruido, axis=1)

print(df['Exposicao_ruido'].value_counts(dropna=False))
print(df['Exposicao_ruido'].isna().sum())

Exposicao_ruido
2    22089
1     2262
Name: count, dtype: int64
0


In [36]:
def marcar_exposicao_lixo(row):
    trabalho = row['E011']
    exposicao = row['M011051']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_lixo'] = df.apply(marcar_exposicao_lixo, axis=1)

print(df['Exposicao_lixo'].value_counts(dropna=False))
print(df['Exposicao_lixo'].isna().sum())

Exposicao_lixo
2    23571
1      780
Name: count, dtype: int64
0


In [37]:
def marcar_exposicao_minerio(row):
    trabalho = row['E011']
    exposicao = row['M011071']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_minerio'] = df.apply(marcar_exposicao_minerio, axis=1)

print(df['Exposicao_minerio'].value_counts(dropna=False))
print(df['Exposicao_minerio'].isna().sum())

Exposicao_minerio
2    22985
1     1366
Name: count, dtype: int64
0


In [38]:
df = df.drop(columns=['M011011'])
df = df.drop(columns=['M011021'])
df = df.drop(columns=['M011051'])
df = df.drop(columns=['M011071'])

### Dor/desconforto no peito

1. N004: dor ou desconforto no peito ao subir uma ladeira, um lance de escadas ou caminhar rápido no plano
2. N005: dor ou desconforto no peito ao caminhar em lugar plano
3. N008: onde é a dor/desconforto no peito

Outros atributos usados: 

4. G059: usa algum aparelho de auxílio para se locomover? (Se G59=1 , siga N10) - incompleto!

In [39]:
def map_dor_peito(row):
    if row['G059'] != 2:
        return 'Não aplicável'  # Ignora quem não respondeu 2 em G059 (736 pessoas)

    if row['N004'] == 2 and row['N005'] == 2:
        return 'Não sente dor'

    if row['N004'] == 1 or row['N005'] == 1:
        dor_map = {
            1: 'Acima ou no meio do peito',
            2: 'Abaixo do peito',
            3: 'Braço esquerdo',
            4: 'Outro lugar'
        }
        return dor_map.get(row['N008'], pd.NA)

    return pd.NA  # Se não cair em nenhum caso anterior 

df['Dor_peito'] = df.apply(map_dor_peito, axis=1)

print(df['Dor_peito'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Dor_peito'].isna().sum()}")

Dor_peito
Não sente dor                20276
Acima ou no meio do peito     2572
Não aplicável                  720
Abaixo do peito                514
Braço esquerdo                 220
Outro lugar                     49
Name: count, dtype: int64
Valores faltantes: 0


In [40]:
df = df.drop(columns=['G059', 'N004', 'N005', 'N008'])

### Exercício físico 

1. P034: Nos últimos três meses, praticou algum tipo de exercício físico ou esporte?
2. P035: Quantos dias por semana.
3. P03701: tempo em horas.
4. P03702: tempo em minutos.

In [41]:
print(df['P035'].max())
print(df['P03701'].max())

7.0
10.0


In [42]:
df['P034'].value_counts()

P034
2.0    16079
1.0     8272
Name: count, dtype: int64

In [43]:
def calcular_e_categorizar_tempo(row):
    # Se não praticou exercícios (resposta 2), atribui 0 horas
    if row['P034'] == 2:
        return 0.0, '0'
    
    # Novo caso: P034 == 1, P035 == 0 e trabalha (E011 = 1, 2 ou 3)
    if row['P034'] == 1 and row['P035'] == 0 and row['E011'] in [1, 2, 3]:
        return 0.0, '0'
    
    # Se praticou e os campos estão preenchidos, calcula tempo total em horas
    if row['P034'] == 1 and pd.notna(row['P035']) and pd.notna(row['P03701']) and pd.notna(row['P03702']):
        minutos_por_dia = row['P03701'] * 60 + row['P03702']
        minutos_semanais = minutos_por_dia * row['P035']
        horas_semanais = minutos_semanais / 60
        horas_semanais = round(horas_semanais, 1)

        # Categoriza por faixa
        if horas_semanais == 0:
            faixa = '0'
        elif horas_semanais <= 5:
            faixa = '0-5h'
        elif horas_semanais <= 10:
            faixa = '5-10h'
        elif horas_semanais <= 20:
            faixa = '10-20h'
        elif horas_semanais <= 40:
            faixa = '20-40h'
        else:
            faixa = '>40h'

        return horas_semanais, faixa

    # Se faltarem dados, retorna valor ausente
    return pd.NA, pd.NA

df[['Tempo_ex_fisico', 'Faixa_ex_fisico']] = df.apply(calcular_e_categorizar_tempo, axis=1, result_type='expand')

print(df['Faixa_ex_fisico'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Faixa_ex_fisico'].isna().sum()}")

Faixa_ex_fisico
0         16220
0-5h       6356
5-10h      1402
10-20h      271
<NA>         69
20-40h       26
>40h          7
Name: count, dtype: int64
Valores faltantes: 69


In [44]:
df = df.drop(columns=['P034', 'P035', 'P03701', 'P03702', 'Tempo_ex_fisico'])

### Hábito de fumar / tabagismo

1. P050: Atualmente, o(a) Sr(a) fuma algum produto do tabaco? 
2. P051: E no passado, o(a) Sr(a) fumou algum produto do tabaco diariamente?
3. P052: E no passado, o(a) Sr(a) fumou algum produto do tabaco?
4. P053:  Que idade o(a) Sr(a) tinha quando começou a fumar produto do tabaco diariamente?

    P050 (Fuma atualmente)

1 = Sim, diariamente → usa P053

2 = Sim, menos que diariamente → verifica P051

3 = Não fuma atualmente → verifica P052

    P051 (Fumou diariamente no passado)

1 = Sim → usa P053

2 = Não → considera como não fumante diário

    P052 (Já fumou alguma vez)

1 = Sim → usa P053

2 ou 3 = Não fumou

In [45]:
def categorizar_inicio_fumo(row):
    p50 = row['P050']
    p51 = row['P051']
    p52 = row['P052']
    p53 = row['P053']  # idade de início

    # Fuma diariamente atualmente
    if p50 == 1:
        idade = p53

    # Fuma menos que diariamente → verifica se já fumou diariamente
    elif p50 == 2:
        if p51 == 1:
            idade = p53
        else:
            return 'Não fuma' 

    # Não fuma atualmente → verifica se já fumou algum dia
    elif p50 == 3:
        if p52 == 1:
            idade = p53
        else:
            return 'Não fuma'

    else:
        return 'Sem informação'  # Resposta inválida ou faltante

    # Categoriza com base na idade de início
    if pd.notna(idade):
        if idade < 18:
            return 'Início antes dos 18'
        elif 18 <= idade <= 30:
            return 'Início entre 18 e 30'
        elif idade > 30:
            return 'Início após os 30'
    return 'Sem informação'  # idade ausente

df['Inicio_tabagismo'] = df.apply(categorizar_inicio_fumo, axis=1)

print(df['Inicio_tabagismo'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Inicio_tabagismo'].isna().sum()}")


Inicio_tabagismo
Não fuma                14010
Início antes dos 18      6739
Início entre 18 e 30     3228
Início após os 30         374
Name: count, dtype: int64
Valores faltantes: 0


 P59. Há quanto tempo o(a) Sr(a) parou de fumar?

5. P05901 - Anos
6. P05902 - Meses
7. P05903 - Semanas
8. P05904 - Dias

In [46]:
def tempo_parou_fumar(row):
    # Fumante atual
    if row['P050'] in [1, 2]:
        return 'Fumante atual'

    # Nunca fumou
    if row['P052'] == 3:
        return 'Nunca fumou'

    # Ex-fumante: calcular tempo total em meses
    if row['P052'] in [1, 2]:
        anos = row.get('P05901', 0) or 0
        meses = row.get('P05902', 0) or 0
        semanas = row.get('P05903', 0) or 0
        dias = row.get('P05904', 0) or 0

        total_meses = anos * 12 + meses + (semanas / 4.345) + (dias / 30)

        if pd.isna(total_meses):
            return pd.NA

        if total_meses < 1:
            return 'Parou há <1 mês'
        elif total_meses < 6:
            return '1-6 meses'
        elif total_meses < 12:
            return '6-12 meses'
        elif total_meses < 60:
            return '1-5 anos'
        else:
            return '>5 anos'

    return pd.NA


df['Tempo_parou_fumar'] = df.apply(tempo_parou_fumar, axis=1)

print(df['Tempo_parou_fumar'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Tempo_parou_fumar'].isna().sum()}")

Tempo_parou_fumar
Nunca fumou        13025
>5 anos             7045
Fumante atual       3233
1-5 anos             827
1-6 meses            134
6-12 meses            65
Parou há <1 mês       22
Name: count, dtype: int64
Valores faltantes: 0


9. Cigarros industrializados (P05401)
10. Cigarros de palha ou enrolados (P05404)
11. Cigarros de cravo ou Bali (P05407)
12. Cachimbo (P05410)
13. Charutos ou cigarrilhas (P05413)
14. Narguilé (P05416)
15. Outro (P05419)

In [47]:
# Dicionário de conversão de respostas (média ponderada semanal)
pesos_fumo = {
    1: 7.0,   # diariamente
    2: 3.0,   # semanalmente
    3: 1.0,   # menos de 1 vez/semana
    4: 0.3,   # menos de 1 por mês
    5: 0.0    # não fuma
}

# Lista dos atributos de frequência
colunas_fumo = ['P05401', 'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419']

# Substituir as categorias pelos pesos
for col in colunas_fumo:
    df[col + '_peso'] = df[col].map(pesos_fumo)

# Somar os pesos como uma média ponderada de intensidade semanal
df['Intensidade_fumo'] = df[[col + '_peso' for col in colunas_fumo]].sum(axis=1)

# Ajustar para quem nunca fumou (do campo anterior, que chamamos de Inicio_tabagismo)
df.loc[df['Inicio_tabagismo'] == 'Nunca fumou', 'Intensidade_fumo'] = 0.0

# (Opcional) arredondar para facilitar leitura
df['Intensidade_fumo'] = df['Intensidade_fumo'].round(1)

print(df['Intensidade_fumo'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Intensidade_fumo'].isna().sum()}")


Intensidade_fumo
0.0     21118
7.0      2509
14.0      266
3.0       199
8.0        63
7.3        53
1.0        49
10.0       39
0.3        20
6.0        12
2.0         5
21.0        5
4.0         3
0.6         2
3.3         2
21.3        1
9.0         1
0.9         1
7.9         1
14.3        1
7.6         1
Name: count, dtype: int64
Valores faltantes: 0


In [48]:
df = df.drop(columns=['P050', 'P051', 'P052', 'P053', 'P05401', 'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419', 'P05901', 'P05902', 'P05903', 'P05904', 
                      'P05401_peso', 'P05404_peso', 'P05407_peso', 'P05410_peso', 'P05413_peso', 'P05416_peso', 'P05419_peso'])

### Atributos sobre doenças cardíacas (módulo Q)

In [49]:
df.loc[df['Q00101'] == 6, 'Q00201'] = 2 #diagnóstico de hipertensão
df.loc[df['Q02901'] == 6, 'Q03001'] = 2 #diagnóstico de diabetes
df.loc[df['Q05901'] == 6, 'Q060'] = 2 #diagnóstico de colesterol alto

In [50]:
print(df['Q00201'].isna().sum())
print(df['Q03001'].isna().sum())
print(df['Q060'].isna().sum())

0
0
0


In [51]:
df = df.drop(columns=['Q00101', 'Q02901', 'Q05901'])    

0 = não tem diagnóstico de doença cardíaca!

In [52]:
df.loc[df['Q06306'] == 2, 'Q06307'] = 0
df.loc[df['Q06306'] == 2, 'Q06308'] = 0
df.loc[df['Q06306'] == 2, 'Q06309'] = 0
df.loc[df['Q06306'] == 2, 'Q06310'] = 0
df.loc[df['Q06306'] == 2, 'Q06311'] = 0
df.loc[df['Q06306'] == 2, 'Q064'] = 0
df.loc[df['Q06306'] == 2, 'Q06506'] = 0
df.loc[df['Q06306'] == 2, 'Q06507'] = 0
df.loc[df['Q06306'] == 2, 'Q06508'] = 0
df.loc[df['Q06306'] == 2, 'Q06509'] = 0
df.loc[df['Q06306'] == 2, 'Q06601'] = 0
df.loc[df['Q06306'] == 2, 'Q067'] = 0

In [53]:
print(df['Q06307'].isna().sum())
print(df['Q06308'].isna().sum())
print(df['Q06309'].isna().sum())
print(df['Q06310'].isna().sum())
print(df['Q06311'].isna().sum())
print(df['Q064'].isna().sum())
print(df['Q06506'].isna().sum())
print(df['Q06507'].isna().sum())
print(df['Q06508'].isna().sum())
print(df['Q06509'].isna().sum())
print(df['Q06601'].isna().sum())
print(df['Q067'].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0


### Preenchendo aqui os atributos H003, H004 e H010

In [54]:
df.loc[df['H001'].isin([4, 5]), 'H002'] = 0

In [55]:
print(df['H002'].isna().sum())

0


In [56]:
df.loc[(df['H002'].isin([1, 0])), 'H003'] = 0

In [57]:
print(df['H003'].isna().sum())

0


In [58]:
df.loc[df['H003'] == 0, 'H004'] = 0

In [59]:
print(df['H004'].isna().sum())

0


In [60]:
df.loc[df['H004'] != 1, 'H010'] = 0

In [61]:
print(df['H010'].isna().sum())

0


### Hábito de beber (alcoolismo) - incompleto (só checar)

1. P027: Com que frequência costuma consumir alguma bebida alcoólica?
2. P02801: Quantos dias por semana costuma consumir alguma bebida alcoólica?
3. P029:  Em geral, no dia que bebe, quantas doses de bebida alcoólica consome?

In [62]:
print(df['P02801'].max()) # = 7 dias
print(df['P029'].max()) # = 60 doses

# → Máximo teórico de doses por semana = 7 * 60 = 420 doses

7.0
60.0


In [63]:
def calcular_doses_semanais(row):
    freq = row['P027']
    dias_semana = row['P02801']
    doses_por_dia = row['P029']

    if freq == 1:
        return 0  # Não bebe nunca
    elif freq == 2:
        return doses_por_dia if pd.notna(doses_por_dia) else pd.NA
    elif freq == 3:
        if pd.notna(dias_semana) and pd.notna(doses_por_dia):
            return dias_semana * doses_por_dia
        else:
            return pd.NA
    return pd.NA

def categorizar_doses(doses):
    if pd.isna(doses):
        return pd.NA
    elif doses == 0:
        return '0 doses'
    elif doses <= 5:
        return '1–5 doses'
    elif doses <= 14:
        return '6–14 doses'
    elif doses <= 30:
        return '15–30 doses'
    elif doses <= 60:
        return '31–60 doses'
    else:
        return '61+ doses'

df['Doses_semanais'] = df.apply(calcular_doses_semanais, axis=1)
df['Quant_doses_alcool_semanal'] = df['Doses_semanais'].apply(categorizar_doses)


print(df['Quant_doses_alcool_semanal'].value_counts(dropna=False))
print("Valores faltantes:", df['Quant_doses_alcool_semanal'].isna().sum())


Quant_doses_alcool_semanal
0 doses        17191
1–5 doses       4070
6–14 doses      2008
15–30 doses      785
31–60 doses      218
61+ doses         79
Name: count, dtype: int64
Valores faltantes: 0


In [64]:
df = df.drop(columns=['P027','P02801', 'P029'])

### Lalala

In [65]:
df.drop(columns=['E011', 'V0001', 'V0024', 'UPA_PNS', 'V0006_PNS'])

,C00301_x,V0026,V0031,B001,C006,C008,C009,C011,E006011,E008,...,Exposicao_ruido,Exposicao_lixo,Exposicao_minerio,Dor_peito,Faixa_ex_fisico,Inicio_tabagismo,Tempo_parou_fumar,Intensidade_fumo,Doses_semanais,Quant_doses_alcool_semanal
0,1.0,1,1,1.0,1.0,81.0,4.0,3.0,0.0,0.0,...,2,2,2,Não sente dor,5-10h,Início antes dos 18,>5 anos,0.0,0.0,0 doses
1,1.0,1,1,2.0,2.0,54.0,4.0,3.0,0.0,0.0,...,2,2,2,Não sente dor,0,Não fuma,Nunca fumou,0.0,0.0,0 doses
2,1.0,1,1,2.0,2.0,41.0,2.0,1.0,0.0,0.0,...,2,2,2,Não sente dor,0-5h,Não fuma,>5 anos,0.0,15.0,15–30 doses
3,1.0,1,1,2.0,1.0,62.0,1.0,2.0,0.0,0.0,...,2,2,2,Não sente dor,0,Início antes dos 18,>5 anos,0.0,8.0,6–14 doses
4,2.0,1,1,2.0,2.0,51.0,4.0,1.0,0.0,0.0,...,2,2,2,Não sente dor,5-10h,Não fuma,Nunca fumou,0.0,0.0,0 doses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24346,1.0,2,1,2.0,2.0,51.0,1.0,4.0,0.0,0.0,...,2,2,2,Acima ou no meio do peito,0,Não fuma,Nunca fumou,0.0,7.0,6–14 doses
24347,2.0,2,1,2.0,1.0,44.0,2.0,4.0,0.0,0.0,...,2,2,2,Não sente dor,0,Não fuma,Nunca fumou,0.0,0.0,0 doses
24348,2.0,2,1,2.0,2.0,54.0,4.0,1.0,0.0,0.0,...,1,2,2,Não sente dor,0,Início entre 18 e 30,Fumante atual,7.0,3.0,1–5 doses
24349,2.0,2,1,3.0,1.0,44.0,4.0,2.0,0.0,0.0,...,1,1,2,Não sente dor,0,Início após os 30,Fumante atual,7.0,24.0,15–30 doses


In [66]:
df.to_csv('../Bases/Criados/base_preenchida.csv', index=False)

In [67]:
df.columns[df.isnull().any()]

Index(['F001021', 'F007021', 'F008021', 'VDF00102', 'I004', 'I00401', 'I00402',
       'I00403', 'M00203', 'P00104', 'P00404', 'E011', 'I00103', 'I001031',
       'P00103', 'P00403', 'Faixa_ex_fisico'],
      dtype='object')

In [68]:
(df.isnull().sum() > 0).sum()

np.int64(17)